# Homework 12.3 - Coding

This is the coding portion of the homework assignment for Section 12.3

In [21]:
from typing import Callable
import numpy as np
from jax import grad
from jax import numpy as jnp


In [22]:
def newton_min(
    f: Callable[[float], float],
    x0: float,
    eps: float = 1e-2,
    maxiter: int = 30
) -> float:
    """Approximates the minimizer of a callable scalar function
    f: R -> R using Newton's method for finding critical points:
    
    x_{k+1} = x_{k} - (f'(x_{k})/f''(x_{k}))

    This method assumes convergence when 
    |x_{k+1} - x_{k}| < eps 

    Raises a RuntimeError if the algorithm does not converge
    within a given number of iterations.

    Args:
        f (Callable[[float], float]): A callable function f:R -> R 
            which we would like to minimize 
        x0 (float): An initial guess for a minimizer 
        eps (float): An error tolerance for successive iterations
            to determine convergence:
            |x_{k+1} - x_{k}| < eps 
            Defaults to 0.01
        maxiter (int): The maximum number of iterations the algorithm
            should be allowed to run before throwing an error 
            indicating a lack of convergence. 
            Defaults to 30
    
    Returns:
        float: A close approximation to a critical point of f
    """
    # Get f' and f'' as callable functions
    df = grad(f)
    d2f = grad(df)

    # Newton iteration for finding critical points 
    x_k = x0 
    for _ in range(maxiter):
        x_kplus1 = x_k - (df(x_k)/d2f(x_k))
        if abs(x_kplus1 - x_k) < eps:
            return x_kplus1 
        x_k = x_kplus1
    return x_kplus1

## Problem 12.15 
Consider the quadratic function 
$$f(\mathbf{x}) = \frac{1}{2} \mathbf{x}^\top Q \mathbf{x} - \mathbf{b}^\top \mathbf{x} + c$$

In the function `quad_grad_descent()`, implement an exact gradient descent method for quadratic functions in the style of $f$, above.

Recall that exact gradient descent for quadratic functions such as $f$ has the iteration scheme
$$\mathbf{x}_{k+1} = \mathbf{x}_{k} - \alpha_k Df(\mathbf{x}_k)^\top$$
where 
$$\alpha_k := \frac{Df(\mathbf{x}_k)Df(\mathbf{x}_k)^\top}{Df(\mathbf{x}_k)Q Df(\mathbf{x}_k)^\top}$$
and 
$$Df(\mathbf{x}_k)^\top = Q\mathbf{x}_k - \mathbf{b}$$

Your code accept the matrix $Q$ and the vector $\mathbf{b}$ from the quadratic equation $f$ above, along with an initial guess $\mathbf{x}_0$ for the minimizer, an error tolerance $\varepsilon$, and a maximum number of iterations, and should return:

1. A close approximation to the local minimizer $\mathbf{x}^*$ of the quadratic function $f$ given above, and
2. The number of iterations used in the algorithm
3. A boolean value indicating whether or not your algorithm converged.

Your code should adhere to the following guidelines:

1. Use the convergence criterion of $\lVert Df(\mathbf{x}_k)^\top \rVert_{2} < \varepsilon$ to know when to terminate the algorithm.
2. If your method fails to converge within the maximum number of iterations, just return the most recent update as the solution, along with the maximum number of iterations as the number of iterations, and `False` for whether the algorithm converged.

In [23]:
def quad_grad_descent(
    Q: np.ndarray,
    b: np.ndarray,
    x0: np.ndarray,
    eps: float = 1e-6,
    maxiter: int = 10000
) -> tuple[np.ndarray, int]:
    """
    A simple implementation of the exact gradient descent
    method to find local minimizers of quadratic functions
    taking the form
    
    f(x) = 1/2 x^T Q x - b^T x + c

    where Q is a positive definite nxn matrix, and b is a length-n vector.

    Uses the convergence criterion
    ||Df(x_{k})^T||_2 < eps

    If the algorithm fails to converge within the given number of iteraions,
    raises a RuntimeError.
    
    Parameters: 
        Q ((n,n) ndarray) - A nxn positive definite matrix used in 
            defining the quadratic function f
        b ((n,) ndarray) - A length n array used in definining
            the quadratic function f
        x0 ((n,) ndarray) - An initial guess (a length-n array)
        eps (float) - A small number representing the error tolerance
            in the 2-norm of the gradient; 
            that is, a number indicating convergence when
            ||Df(x_{k})^T||_2 < eps
        maxiter (int) - The maximum number of iterations the algorithm
            is allowed to go before raising a RuntimeError indicating
            a lack of convergence

    Returns: 
        (n,) ndarray - A close approximation of the local minimizer x* of 
            f(x) = 1/2 x^T Q x - b^T x + c
        int - The number of iterations it took to converge
        bool - Whether or not the algorithm converged to within the
            error tolerance
    """
    x = x0 
    
    for i in range(maxiter + 1):
        Df_x = (Q @ x - b).T

        if np.linalg.norm(Df_x.T, 2) < eps:
            return x, i, True

        alpha = np.dot(Df_x, Df_x) / (Df_x @ Q @ Df_x.T)
        x = x - alpha*Df_x 

    return x, maxiter, False




Now, run the following cell to approximate the minimizer of the function 
$$f(\mathbf{x}) = \begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix} \begin{bmatrix}2 & -1 & 0 \\ -1 & 2 & -1 \\ 0 & -1 & 2  \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} - \begin{bmatrix}2 & 4 & -3 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}$$
which should be $(2.75, 3.5, 0.25)$

In [24]:
# ------------------- DO NOT EDIT. JUST RUN. ------------------- #
Q = np.array([[2,-1,0],[-1,2,-1],[0,-1,2]])
b = np.array([2,4,-3])
x0 = np.array([1,6,3])  # Initial guess for minimizer
min_approx, num_iters, converged = quad_grad_descent(Q, b, x0)
print("Approximate Minimizer: ", min_approx)
print("Number of Iterations of Algorithm: ", num_iters)
print("Converged to Within Tolerance: ", converged)

Approximate Minimizer:  [2.75000052 3.50000096 0.25000052]
Number of Iterations of Algorithm:  40
Converged to Within Tolerance:  True


## Problem 12.16

Now, in the function `exact_grad_descent()`, write up an implementation of exact gradient descent for an arbitrary function $f: \mathbb{R}^n \to \mathbb{R}$. 

Recall that exact gradient descent for an arbitrary function $f: \mathbb{R}^n \to \mathbb{R}$ has the iteration scheme
$$\mathbf{x}_{k+1} = \mathbf{x}_{k} - \alpha_k Df(\mathbf{x}_k)^\top$$
where 
$$\alpha_k := \operatorname{argmin}_{\alpha > 0} \phi_k(\alpha)$$
and
$$\phi_k(\alpha) := f(\mathbf{x}_k - \alpha Df(\mathbf{x}_k)^\top)$$

Your implementation should accept a callable function $f: \mathbb{R}^n \to \mathbb{R}$, an initial guess $\mathbf{x}_0$ for the minimizer, an error tolerance $\varepsilon$, and a maximum number of iterations.

Your code should return:

1. A close approximation to the local minimizer $\mathbf{x}^*$ of the quadratic function $f$ given above, and
2. The number of iterations used in the algorithm
3. A boolean value indicating whether or not your algorithm converged.

Your code should adhere to these guidelines:

1. Assume that all passed-in functions $f$ are coded using `jax`'s version of numpy `jnp` in order to allow for automatic differentiation
2. Before any iteration has happened, use the `jax` function `grad()` to get a callable version of the derivative $Df(\mathbf{x})$ of the provided function $f$.
3. At each update step, use the provided code `newton_min()` (found at the top of this file) to find $\alpha_k$ by minimizing the function $\phi_k(\alpha)$ (HINT: You may need to make a new lambda function at each iteration to pass to `newton_min()`)
4. Use the criterion $\lVert Df(\mathbf{x}_{k})^\top \rVert_{2} < \varepsilon$ to determine convergence.
5. If your method fails to converge within the maximum number of iterations, just return the most recent update as the solution, along with the maximum number of iterations as the number of iterations, and `False` for whether the algorithm converged.

**BEFORE DOING THIS, HOWEVER...**

Note that `newton_min()` takes in an initial guess for your parameter $\alpha$ during the "line search" part of the algorithm in step (3). You must choose a good initial guess to guarantee convergence. Which $\alpha_0$ guess will you choose and why?

(HINT: It should be very, very small to be a good initial guess. Can you think about why?)

**RESPONSE**: _use something small like 10^-6.  This is because the true alpha we're looking for is likely extremely small already, and we want to approach it slowly and directly as opposed to large and zigzaggy_

Now, code up the algorithm as described above:

In [25]:



def exact_grad_descent(
    f: Callable[[jnp.ndarray], float],
    x0: jnp.ndarray,
    eps: float = 1e-6,
    maxiter: int = 1000
) -> tuple[jnp.ndarray, int]:
    """
    An implementation of the exact gradient descent method for any
    arbitrary function f. Estimates a local minimizer of f close to x0.

    Uses the convergence criterion
    ||Df(x_{k})^T||_2 < eps

    If the algorithm fails to converge within the given number of iteraions,
    raises a RuntimeError.
    
    Args: 
        f (Callable[[jnp.ndarray], float]) - An arbitrary
            function to minimize, from R^n to R.
        x0 ((n,) jnp.ndarray) - A length-n jax-compatible
            numpy array representing the initial guess/seed
            for the gradient descent algorithm
        eps (float) - A small number representing the error tolerance
            in the 2-norm of the gradient; 
            that is, a number indicating convergence when
            ||Df(x_{k})^T||_2 < eps
        maxiter (int) - The maximum number of iterations the algorithm
            is allowed to go before raising a RuntimeError indicating
            a lack of convergence
    
    Returns: 
        (n,) jnp.ndarray - A close approximation of the local 
            minimizer x* of f 
        int - The number of iterations used in the algorithm

    Raises:
        RuntimeError - If the algorithm does not converge in the given
            number of iterations
    """

    Df = grad(f)

    x = x0 
    
    for i in range(maxiter + 1):
        Df_x = Df(x)

        if np.linalg.norm(Df_x.T, 2) < eps:
            return x, i, True

        g = lambda alpha: f(x - alpha * Df_x)
        alpha = newton_min(g, 10e-6, eps)
        x = x - alpha*Df_x 

    raise ValueError(f"Did not converge after {maxiter} iterations")



Now, run the following code cell to test the `exact_grad_descent()` function by trying to find the minimizer of
$$f(x,y) = -\cos(x)\cos(y)e^{-(x-\pi)^2 + (y-\pi)^2}$$
using the initial guess $(x_0, y_0) = (2.8, 2.8)$.

The result should be $(x^*, y^*) = (\pi, \pi)$.

In [26]:
# ------------------- DO NOT EDIT. JUST RUN. ------------------- #
x0 = jnp.array([2.9, 2.9])
f = lambda x: -jnp.cos(x[0]) * jnp.cos(x[1]) * jnp.exp(-((x[0] - jnp.pi)**2 + (x[1] - jnp.pi)**2))
estimate, num_iters, converged = exact_grad_descent(f,x0)
print("Estimate for Local Minimizer: ", estimate)
print("Number of Iterations: ", num_iters)
print("Algorithm Converged? ", converged) 

Estimate for Local Minimizer:  [3.1415927 3.1415927]
Number of Iterations:  1
Algorithm Converged?  True


## Problem 12.17

Apply your `exact_grad_descent()` function from the previous problem to the Rosenbrock function
$$f(x,y) = 100(y-x^2)^2 + (1-x)^2$$
with an initial guess of $(x_0, y_0) = (-2,2).$ 

The true minimizer is $(1,1)$. 

Try to see if you can get it to converge to within $10^{-5}$ of the true minimizer.

(Hint: you probably won't without an obscene number of iterations or an insanely small error tolerance - try it to see!)

(This cell may take a while to run for high numbers of iterations/small errors. This is normal and okay - so don't sweat it if you can't make it run for high numbers of iterations quickly.)

In [27]:
# TODO: Edit parameters eps and maxiter in this cell and see what happens.
rosenbrock = lambda x: 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2 
x0 = jnp.array([-2.,2.])
eps = 1e-3             # Play around with this parameter
maxiter = 500           # Play around with this parameter, too
exact_grad_descent(rosenbrock, x0,eps, maxiter)

ValueError: Did not converge after 500 iterations

Whether or not you got it to converge, you'll see that it took a long time to run this algorithm. Why is this? (Hint: Google the Rosenbrock function)

**RESPONSE:** _TODO: The function is designed to be hard to converge.  there's a valley which is easy to find, but the convergence from there to the global minimum is really slow and takes forever._